#### **Codes for the Statistical Analyses in the User Study of LivePoem**

##### Step1: Import Libraries and Result File

*N.B. To protect the privacy of participants, we **anonymised** the result file by filtering the personal information in the first several columns.*

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ks_2samp, mannwhitneyu
import statsmodels.formula.api as smf
from statsmodels.stats.diagnostic import het_breuschpagan
from statsmodels.stats.stattools import durbin_watson

In [2]:
## Load and read the results file
table_path = f"./test_results.csv"
df = pd.read_csv(table_path)

In [3]:
## question ids
before_ids = [5, 10, 15, 20, 25]  ## The question IDs for pre-tests
after_ids_txt = [4, 10, 16, 22, 28] ## The question IDs for post-tests under textbook condition (TB)
after_ids_vis = [3, 8, 13, 18, 23] ## The question IDs for post-tests under storyboard condition (SB)

## block ids
before_blks_txt = [7, 13]  ## The question block IDs for pre-test samples under textbook condition (TB)
before_blks_vis = [9, 11]  ## The question block IDs for pre-test samples under storyboard condition (SB)
before_blks = before_blks_vis + before_blks_txt  ## All the pre-test samples

rating_txt = ["16.2", "16.3", "16.4"]  ## The question IDs for SAM ratings under textbook condition (TB)
rating_vis = ["15.2", "15.3", "15.4"]  ## The question IDs for SAM ratings under storyboard condition (SB)

### The IDs for correct answers for each question
### The dictionary if formatted as:
### Each item is "the ID of the poetry: correct answers for the five questions"
answer_ids = {
    7: [1, 1, 4, 2, 3],
    9: [1, 4, 3, 2, 1],
    11: [2, 3, 1, 1, 4],
    13: [1, 2, 2, 4, 4],
    8: [1, 1, 4, 2, 3],
    10: [1, 4, 3, 2, 1],
    12: [2, 3, 1, 1, 4],
    14: [1, 2, 2, 4, 4],
}

In [4]:
## demographic information
## telephone number is Q2.4
## gender Q2.5 1-male; 2-female; 3-non-binary/third; 4-prefer not to say
## Chinese proficiency Q2.6 1-5
## Music proficiency Q2.7

## Initialise several dictionaries for counting the demographic information
gender = {
    '1': 0,  ## Male
    '2': 0,  ## Female
    '3': 0,  ## Non-binary
    '4': 0,  ## Prefer not to disclose
}
lang_prof = {  ## ILR Scale
    '1': 0,
    '2': 0,
    '3': 0,
    '4': 0,
    '5': 0
}
music_prof = {
    '1': 0,  ## Beginner
    '2': 0,  ## Intermediate
    '3': 0,  ## Advanced
}
users = {}  ## User ID (for data aggregation)
parti_nbr = 0  ## Count the number of participants (i.e., the valid responses)

## Structure the data
for idx, row in df.iterrows(): 
    if row['Finished'] == 0:  ## Filter unfinished questionnaires
        continue
    ## Retrieve the information
    user_id = str(int(row['Q2.4'])).strip()
    if user_id not in users.keys():
        users[user_id] = []
    user_gender = str(int(row['Q2.5']))
    gender[user_gender] += 1
    user_language = str(int(row['Q2.6']))
    lang_prof[user_language] += 1
    user_music = str(int(row['Q2.7']))
    music_prof[user_music] += 1
    users[user_id].extend([user_gender, user_language, user_music])
    parti_nbr += 1

## Print the demographic information
print(f"Gender: {gender}\nLanguage Proficiency: {lang_prof}\nMusic Proficiency: {music_prof}")

Gender: {'1': 14, '2': 10, '3': 0, '4': 1}
Language Proficiency: {'1': 1, '2': 14, '3': 10, '4': 0, '5': 0}
Music Proficiency: {'1': 16, '2': 8, '3': 1}


##### Step2: Computing Test Performance in Pre- and Post-Tests

In [5]:
## Test performance in pre-test (SB condition)
## collect participants accuracy before viewing STORYBOARDS
accuracy_before_vis_user = {}
true_cnt, total_cnt = 0, 0
for idx, row in df.iterrows():
    if row['Finished'] == 0:
        continue
    user_id = str(int(row['Q2.4'])).strip()
    if user_id not in accuracy_before_vis_user.keys():
        accuracy_before_vis_user[user_id] = 0
    if len(users[user_id]) <= 3:
        users[user_id].append({})
    for b_blk_id in before_blks_vis:
        sample_id = f"{b_blk_id}_SB_Bef"
        if sample_id not in users[user_id][3].keys():
            users[user_id][3][sample_id] = []
        for ques_idx, b_ques_id in enumerate(before_ids):
            ans = int(row[f'Q{b_blk_id}.{b_ques_id}'])
            gt = answer_ids[b_blk_id][ques_idx]
            if ans == gt:
                users[user_id][3][sample_id].append(1)
                true_cnt += 1
                accuracy_before_vis_user[user_id] += 1
            else:
                users[user_id][3][sample_id].append(0)
            total_cnt += 1
print("Test performance in pre-test (SB condition)")
accuracy_before_vis_user_list = np.array(list(accuracy_before_vis_user.values()))/10
print(f"Mean: {np.mean(accuracy_before_vis_user_list)}\nSD: {np.std(accuracy_before_vis_user_list)}")


Test performance in pre-test (SB condition)
Mean: 0.6519999999999999
SD: 0.20023985617254125


In [6]:
## Test performance in post-test (SB condition)
## collect participants accuracy post viewing STORYBOARDS
accuracy_after_vis_user = {}
true_cnt, total_cnt = 0, 0
for idx, row in df.iterrows():
    if row['Finished'] == 0:
        continue
    user_id = str(int(row['Q2.4'])).strip()
    if user_id not in accuracy_after_vis_user.keys():
        accuracy_after_vis_user[user_id] = 0
    if len(users[user_id]) <= 3:
        users[user_id].append({})
    for b_blk_id in before_blks_vis:
        sample_id = f"{b_blk_id}_SB_Aft"
        if sample_id not in users[user_id][3].keys():
            users[user_id][3][sample_id] = []
        for ques_idx, a_ques_id in enumerate(after_ids_vis):
            ans = int(row[f'Q{b_blk_id+1}.{a_ques_id}'])
            gt = answer_ids[b_blk_id+1][ques_idx]
            if ans == gt:
                users[user_id][3][sample_id].append(1)
                true_cnt += 1
                accuracy_after_vis_user[user_id] += 1
            else:
                users[user_id][3][sample_id].append(0)
            total_cnt += 1
print(f"Test performance in post-test (SB condition)")
accuracy_after_vis_user_list = np.array(list(accuracy_after_vis_user.values()))/10
print(f"Mean: {np.mean(accuracy_after_vis_user_list)}\nSD: {np.std(accuracy_after_vis_user_list)}")

Test performance in post-test (SB condition)
Mean: 0.7560000000000001
SD: 0.12354756169184403


In [7]:
import numpy as np
import scipy.stats as stats

print("Statistical test results (SB condition)")
# Example data
accuracy_before = np.array(accuracy_before_vis_user_list)  # Replace with your accuracy data before the test
accuracy_after = np.array(accuracy_after_vis_user_list)   # Replace with your accuracy data after the test

# Check normality of differences
differences = accuracy_after - accuracy_before
stat, p_value = stats.shapiro(differences)
print(f'Shapiro-Wilk Test statistic: {stat}, p-value: {p_value}')

# Perform Paired t-Test
t_stat, p_value = stats.ttest_rel(accuracy_after, accuracy_before)
print(f'Paired t-Test statistic: {t_stat}, p-value: {p_value}')

# Perform Wilcoxon signed-rank test
# stat, p_value = stats.wilcoxon(accuracy_after - accuracy_before)
stat, p_value = stats.wilcoxon(accuracy_after_vis_user_list - accuracy_before_vis_user_list)
print(f'Wilcoxon signed-rank test statistic: {stat}, p-value: {p_value}')

Statistical test results (SB condition)
Shapiro-Wilk Test statistic: 0.9139395379488897, p-value: 0.03735121659857334
Paired t-Test statistic: 3.2105290566166693, p-value: 0.0037439828228677838
Wilcoxon signed-rank test statistic: 52.0, p-value: 0.014920784493116672


In [8]:
## Test performance in pre-test (TB condition)
## collect participants accuracy before viewing TEXTBOOKS
true_cnt, total_cnt = 0, 0
accuracy_before_txt_user = {}
for idx, row in df.iterrows():
    if row['Finished'] == 0:
        continue
    user_id = str(int(row['Q2.4'])).strip()
    if user_id not in accuracy_before_txt_user.keys():
        accuracy_before_txt_user[user_id] = 0
    if len(users[user_id]) <= 3:
        users[user_id].append({})
    for b_blk_id in before_blks_txt:
        sample_id = f"{b_blk_id}_TB_Bef"
        if sample_id not in users[user_id][3].keys():
            users[user_id][3][sample_id] = []
        for ques_idx, b_ques_id in enumerate(before_ids):
            ans = int(row[f'Q{b_blk_id}.{b_ques_id}'])
            gt = answer_ids[b_blk_id][ques_idx]
            if ans == gt:
                users[user_id][3][sample_id].append(1)
                accuracy_before_txt_user[user_id] += 1
                true_cnt += 1
            else:
                users[user_id][3][sample_id].append(0)
            total_cnt += 1

print(f"Test performance in pre-test (TB condition)")
accuracy_before_txt_user_list = np.array(list(accuracy_before_txt_user.values()))/10
print(f"Mean: {np.mean(accuracy_before_txt_user_list)}\nSD: {np.std(accuracy_before_txt_user_list)}")

Test performance in pre-test (TB condition)
Mean: 0.6
SD: 0.2513961017995307


In [9]:
## Test performance in post-test (TB condition)
## collect participants accuracy post viewing TEXTBOOKS
true_cnt, total_cnt = 0, 0
accuracy_after_txt_user = {}
for idx, row in df.iterrows():
    if row['Finished'] == 0:
        continue
    user_id = str(int(row['Q2.4'])).strip()
    if user_id not in accuracy_after_txt_user.keys():
        accuracy_after_txt_user[user_id] = 0
    if len(users[user_id]) <= 3:
        users[user_id].append({})
    for b_blk_id in before_blks_txt:
        sample_id = f"{b_blk_id}_TB_Aft"
        if sample_id not in users[user_id][3].keys():
            users[user_id][3][sample_id] = []
        for ques_idx, a_ques_id in enumerate(after_ids_txt):
            ans = int(row[f'Q{b_blk_id+1}.{a_ques_id}'])
            gt = answer_ids[b_blk_id+1][ques_idx]
            if ans == gt:
                users[user_id][3][sample_id].append(1)
                accuracy_after_txt_user[user_id] += 1
                true_cnt += 1
            else:
                users[user_id][3][sample_id].append(0)
            total_cnt += 1
    users[user_id].extend([int(row['Q15.2_1']), 
                           int(row['Q15.3_1']), 
                           int(row['Q15.4_1']),
                           int(row['Q16.2_1']), 
                           int(row['Q16.3_1']), 
                           int(row['Q16.4_1']),])

print(f"Test performance in post-test (TB condition)")
accuracy_after_txt_user_list = np.array(list(accuracy_after_txt_user.values()))/10
print(f"Mean: {np.mean(accuracy_after_txt_user_list)}\nSD: {np.std(accuracy_after_txt_user_list)}")

Test performance in post-test (TB condition)
Mean: 0.8080000000000002
SD: 0.23137847782367313


##### Step3: Performing Linear Mixed-Effects Model (LMM) on Aggregated Data Points

In [10]:
## Structure data points to meet the format requirements of LMM

# Prepare an empty list to store rows
rows = []

id2difficulty = {7: 8, 9: 1, 11: 6, 13: 2}
id2order = {7: 1, 9: 2, 11: 3, 13: 4}

poem_conds = {"9_SB", "11_SB", "7_TB", "13_TB"}

# Loop through the dictionary
for user_id, details in users.items():
    gender, language_proficiency, music_proficiency, _, p_sb, a_sb, d_sb, p_tb, a_tb, d_tb = details

    for poem_cond in poem_conds:
        poem_id, material = poem_cond.split("_")
        acc_bef = sum(list(users[user_id][3][f"{poem_cond}_Bef"]))/5
        acc_aft = sum(list(users[user_id][3][f"{poem_cond}_Aft"]))/5
        difficulty = id2difficulty[int(poem_id)]
        order = id2order[int(poem_id)]
        if material == "TB":
            pleasure, arousal, dominance = p_tb, a_tb, d_tb
        elif material == "SB":
            pleasure, arousal, dominance = p_sb, a_sb, d_sb
        else:
            print("OUT OF BOUNDARY")

        rows.append([user_id, gender, language_proficiency, music_proficiency, 
                     f"{material}", int(difficulty), acc_bef, acc_aft, order, pleasure, arousal, dominance])

# Convert list to DataFrame
df_long = pd.DataFrame(rows, 
    columns=["participant", "gender", "language_proficiency", "music_proficiency", 
             "condition", "difficulty", "accuracy_before", "accuracy_after", "order", "pleasure", "arousal", "dominance"])

# Compute improvement score
df_long["improvement"] = df_long["accuracy_after"] - df_long["accuracy_before"]

# Display the transformed DataFrame
print(df_long)

   participant gender language_proficiency music_proficiency condition  \
0            1      1                    2                 2        TB   
1            1      1                    2                 2        SB   
2            1      1                    2                 2        TB   
3            1      1                    2                 2        SB   
4            2      1                    2                 1        TB   
..         ...    ...                  ...               ...       ...   
95          28      1                    2                 2        SB   
96          29      2                    1                 1        TB   
97          29      2                    1                 1        SB   
98          29      2                    1                 1        TB   
99          29      2                    1                 1        SB   

    difficulty  accuracy_before  accuracy_after  order  pleasure  arousal  \
0            8              0.6   

In [11]:
np.random.seed(42)
# Create the improvement column if not already created
df_long["improvement"] = df_long["accuracy_after"] - df_long["accuracy_before"]
# df_long['condition'] = pd.Categorical(df_long['condition'], categories=['TB', 'SB'], ordered=True)

# Mixed linear model for improvement including all relevant factors
# Dependent variable: improvement
# Independent variables: test condition (interacts with) pre-test acc, language proficiency, difficulty level of poems, order of poems
model_combined = smf.mixedlm(
    "improvement ~ condition * accuracy_before + language_proficiency + difficulty + order",  # Outcome: improvement
    df_long,
    groups=df_long["participant"],  # Random intercept for each participant
    re_formula="1"  # Random intercept for each participant
)

# Fit the LMM model
result_combined = model_combined.fit()

# Save the residuals of LMM to perform assumption diagnosis
residuals = np.array(result_combined.resid)

# Display the results
print(result_combined.summary())

## Assumption diagnoses

# 1. **Normality of Residuals**
# Shapiro-Wilk test for normality
shapiro_test = stats.shapiro(residuals)
print(f"Shapiro-Wilk test p-value: {shapiro_test}")

# 2. **Homogeneity of Variance (Homoscedasticity)**
# Breusch-Pagan test for homoscedasticity
# _, pval, _, _ = het_breuschpagan(residuals, result_combined.model.exog)
het_results = het_breuschpagan(residuals, result_combined.model.exog)
print(f"Breusch-Pagan test p-value: {het_results}")

# 3. **Independence of Residuals**
# Durbin-Watson test for autocorrelation
dw_stat = durbin_watson(residuals)
print(f"Durbin-Watson statistic: {dw_stat}")

# 4. **Random Effects Structure**
# Extract random effects variance
random_effect_variance = result_combined.cov_re.iloc[0, 0]
print(f"Random effect variance: {random_effect_variance}")

                  Mixed Linear Model Regression Results
Model:                   MixedLM      Dependent Variable:      improvement
No. Observations:        100          Method:                  REML       
No. Groups:              25           Scale:                   0.0292     
Min. group size:         4            Log-Likelihood:          10.0456    
Max. group size:         4            Converged:               Yes        
Mean group size:         4.0                                              
--------------------------------------------------------------------------
                                Coef.  Std.Err.   z    P>|z| [0.025 0.975]
--------------------------------------------------------------------------
Intercept                        0.959    0.197  4.868 0.000  0.573  1.345
condition[T.TB]                  0.011    0.092  0.123 0.902 -0.169  0.192
language_proficiency[T.2]       -0.250    0.138 -1.811 0.070 -0.521  0.021
language_proficiency[T.3]       -0.192    0.

/opt/anaconda3/envs/myenv/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
